In [1]:
import pandas as pd
import numpy as np
from plotnine import *

c:\Users\wafia\miniconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5


In [5]:
# Opening population data
df_pop = pd.read_csv('../20_intermediate_files/vital_stats_merged.csv', index_col=0)
df_pop = df_pop.drop('Crude Rate', axis = 1)
df_pop['county_name_only'] = df_pop["County Name"].str[:-7]
df_pop.head(2)

,Year,State,State Code,County,County Code,Cause of death,Cause of death Code,Deaths,Population,State_Code,County Name,county_name_only
32,2007,Arkansas,5.0,"Garland County, AR",5051.0,Poisoning by and exposure to other and unspeci...,Y14,12.0,94753.0,AR,Garland County,Garland
33,2007,Arkansas,5.0,"Pulaski County, AR",5119.0,Poisoning by and exposure to other and unspeci...,Y14,11.0,373403.0,AR,Pulaski County,Pulaski


In [6]:
# Random checking to make sure we have the states
(df_pop['State']=='Washington').value_counts()

False    1984
True      117
Name: State, dtype: int64

In [7]:
# Opening perscription data
df_pres = pd.read_csv('../20_intermediate_files/prescription_data_grouped_1.csv')
(df_pres['MME']==0).value_counts()
df_pres["T_YEAR"].unique()

array([2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014], dtype=int64)

In [8]:
# Dropping unnecessary columns
df_pop = df_pop.drop(["State Code", "County", 'County Code', 'Cause of death', 'Cause of death Code', 'County Name'], axis=1)

In [9]:
df_pop.sample(2)

,Year,State,Deaths,Population,State_Code,county_name_only
832,2008,California,12.0,408972.0,CA,Solano
4044,2012,California,10.0,178586.0,CA,Shasta


In [10]:
# Creating filter column for population df
df_pop_grouped = df_pop.groupby(['Year', 'State', 'State_Code', 'county_name_only']).sum().reset_index()
df_pop_grouped['county_name_only'] = df_pop_grouped['county_name_only'].str.lower()
df_pop_grouped['Year_string'] = df_pop_grouped['Year'].astype('str')
df_pop_grouped['filter'] = df_pop_grouped['State_Code'] + df_pop_grouped['county_name_only'] + df_pop_grouped['Year_string']
df_pop_grouped

,Year,State,State_Code,county_name_only,Deaths,Population,Year_string,filter
0,2007,Arkansas,AR,garland,12.0,94753.0,2007,ARgarland2007
1,2007,Arkansas,AR,pulaski,11.0,373403.0,2007,ARpulaski2007
2,2007,Arkansas,AR,sebastian,21.0,245892.0,2007,ARsebastian2007
3,2007,California,CA,alameda,178.0,5822860.0,2007,CAalameda2007
4,2007,California,CA,butte,51.0,434938.0,2007,CAbutte2007
...,...,...,...,...,...,...,...,...
986,2012,Washington,WA,snohomish,121.0,2199108.0,2012,WAsnohomish2012
987,2012,Washington,WA,spokane,67.0,951470.0,2012,WAspokane2012
988,2012,Washington,WA,thurston,22.0,516664.0,2012,WAthurston2012
989,2012,Washington,WA,yakima,15.0,246977.0,2012,WAyakima2012


In [11]:
# Filtering year and lowercase county name
df_pres = df_pres.loc[(df_pres['T_YEAR']>2006) & (df_pres['T_YEAR']<2013)]
df_pres['county_name_only'] = df_pres['BUYER_COUNTY'].str.lower()

In [12]:
# Grouping prescription df to avoid duplicate state and county name
df_pres_grouped = df_pres.groupby(['BUYER_STATE', 'BUYER_COUNTY', 'T_YEAR', 'county_name_only']).sum().reset_index()
df_pres_grouped

,BUYER_STATE,BUYER_COUNTY,T_YEAR,county_name_only,MME
0,AR,ACCOMACK,2007,accomack,0.0
1,AR,ACCOMACK,2008,accomack,0.0
2,AR,ACCOMACK,2009,accomack,0.0
3,AR,ACCOMACK,2010,accomack,0.0
4,AR,ACCOMACK,2011,accomack,0.0
...,...,...,...,...,...
144667,WY,ZAVALA,2008,zavala,0.0
144668,WY,ZAVALA,2009,zavala,0.0
144669,WY,ZAVALA,2010,zavala,0.0
144670,WY,ZAVALA,2011,zavala,0.0


In [ ]:
# Creating filter column for population df
df_pres_grouped['Year'] = df_pres_grouped['T_YEAR'].astype('str')
df_pres_grouped['filter'] = df_pres_grouped['BUYER_STATE'] + df_pres_grouped['county_name_only'] + df_pres_grouped['Year']
df_pres_grouped.loc[df_pres_grouped['BUYER_STATE']=='FL']


In [ ]:
# filtering the data to only county and state exist in population data
df_pres_grouped.loc[df_pres_grouped['filter'].isin(df_pop_grouped['filter']),:]

In [ ]:
# dropping unnecessary columns
df_pop_grouped = df_pop_grouped.drop(['Year_string', 'filter'], axis=1)
df_pres_grouped = df_pres_grouped.drop(['Year', 'filter'], axis=1)

In [ ]:
# Merge both dataframe into merged_df
merged_df = df_pop_grouped.merge(df_pres_grouped, left_on=["Year","State_Code","county_name_only"], right_on=["T_YEAR","BUYER_STATE","county_name_only"])
merged_df = merged_df.drop(["BUYER_STATE", "BUYER_COUNTY", "T_YEAR"], axis=1)

In [ ]:
# Checking value below 10 (small value of mme seems irrelevant)
merged_df.loc[merged_df['MME']<10,:]

In [ ]:
# defining control states
we_want_texas = ['Arkansas', 'California', 'Georgia', 'Missouri', 'New York', 'Wyoming']
we_want_washington = ['Hawaii', 'Iowa', 'Kansas', 'Maine', 'Massachusetts',
       'Minnesota', 'Montana', 'Nebraska', 'North Dakota', 'Oregon',
       'South Dakota', 'Virginia', 'Wyoming']
we_want_florida = ['Nevada', 'New York', 'California']

df_washington = merged_df[merged_df['State'].isin(we_want_washington)]
df_texas = merged_df[merged_df['State'].isin(we_want_texas)]
df_florida = merged_df[merged_df['State'].isin(we_want_florida)]

In [ ]:
df_washington

In [ ]:
# Grouping control state
groupedby_df_washington = df_washington.groupby(["Year"]).sum() 
groupedby_df_washington = groupedby_df_washington.reset_index(level=None, drop=False, inplace=False, col_level=0, col_fill='')
groupedby_df_washington['Prescription_Rate'] = (groupedby_df_washington['MME']/groupedby_df_washington['Population'])*100_000

groupedby_df_texas = df_texas.groupby(["Year"]).sum() # we feel that sum is the most accurate function to use
groupedby_df_texas = groupedby_df_texas.reset_index(level=None, drop=False, inplace=False, col_level=0, col_fill='')
groupedby_df_texas['Prescription_Rate'] = (groupedby_df_texas['MME']/groupedby_df_texas['Population'])*100_000

groupedby_df_florida = df_florida.groupby(["Year"]).sum() # we feel that sum is the most accurate function to use
groupedby_df_florida = groupedby_df_florida.reset_index(level=None, drop=False, inplace=False, col_level=0, col_fill='')
groupedby_df_florida['Prescription_Rate'] = (groupedby_df_florida['MME']/groupedby_df_florida['Population'])*100_000

groupedby_df_washington['Policy_Change'] = "Control"
groupedby_df_florida['Policy_Change'] = "Control"
groupedby_df_texas['Policy_Change'] = "Control"

In [ ]:
groupedby_df_florida

In [ ]:
# Grouping treatment state
we_want_only_texas = ['Texas']
we_want_only_washington = ['Washington']
we_want_only_florida = ['Florida']

df_only_washington = merged_df[merged_df['State'].isin(we_want_only_washington)]
df_only_texas = merged_df[merged_df['State'].isin(we_want_only_texas)]
df_only_florida = merged_df[merged_df['State'].isin(we_want_only_florida)]

groupedby_df_only_washington = df_only_washington.groupby(["Year"]).sum() 
groupedby_df_only_washington = groupedby_df_only_washington.reset_index(level=None, drop=False, inplace=False, col_level=0, col_fill='')
groupedby_df_only_washington['Prescription Rate'] = (groupedby_df_only_washington['MME']/groupedby_df_only_washington['Population'])*100_000

groupedby_df_only_texas = df_only_texas.groupby(["Year"]).sum() 
groupedby_df_only_texas = groupedby_df_only_texas.reset_index(level=None, drop=False, inplace=False, col_level=0, col_fill='')
groupedby_df_only_texas['Prescription Rate'] = (groupedby_df_only_texas['MME']/groupedby_df_only_texas['Population'])*100_000

groupedby_df_only_florida = df_only_florida.groupby(["Year"]).sum() 
groupedby_df_only_florida = groupedby_df_only_florida.reset_index(level=None, drop=False, inplace=False, col_level=0, col_fill='')
groupedby_df_only_florida['Prescription Rate'] = (groupedby_df_only_florida['Deaths']/groupedby_df_only_florida['Population'])*100_000

groupedby_df_only_washington['Policy_Change'] = "Treatment"
groupedby_df_only_texas['Policy_Change'] = "Treatment"
groupedby_df_only_florida['Policy_Change'] = "Treatment"

In [ ]:
groupedby_df_only_florida

In [ ]:
# function to make ggplot
def diffIndiff(
    prepolicy_contr,
    postpolicy_contr,
    prepolicy_treatment,
    postpolicy_treatment,
    xvar,
    yvar,
    policyyear,
):
    dd = (
        ggplot()
        # plot all chosen states,  pre policy year
        + geom_smooth(
            prepolicy_contr,
            aes(x=xvar, y=yvar, color="Policy_Change"),
            method="lm",
        )
        # plot all chosen states, post policy year
        + geom_smooth(
            postpolicy_contr,
            aes(x=xvar, y=yvar, color="Policy_Change"),
            method="lm",
        )
        # plot treatment, pre policy year
        + geom_smooth(
            prepolicy_treatment,
            aes(x=xvar, y=yvar, color="Policy_Change"),
            method="lm",
        )
        # plot treatment, post policy year
        + geom_smooth(
            postpolicy_treatment,
            aes(x=xvar, y=yvar, color="Policy_Change"),
            method="lm",
        )
        + geom_vline(xintercept=policyyear, linetype="dotted")
        + xlab("Year")
        + theme_classic(base_family="Times")
        + scale_x_continuous(breaks=[2007, 2008, 2009, 2010, 2011, 2012], limits=[2008, 2012])
    )
    return dd

def pre_post(prepolicy_treatment, postpolicy_treatment, xvar, yvar, policyyear):
    ppo = (
        ggplot()
        # plot treatment, pre policy year
        + geom_smooth(
            prepolicy_treatment,
            aes(x=xvar, y=yvar),
            method="lm",
        )
        # plot treatment, post policy year
        + geom_smooth(
            postpolicy_treatment,
            aes(x=xvar, y=yvar),
            method="lm",
        )
        + geom_vline(xintercept=policyyear, linetype="dotted")
        + xlab("Year")
        +scale_color_manual(values=["darkturquoise","darkturquoise"])
        + theme_classic(base_family="Times")
        + scale_x_continuous(breaks=[2007, 2008, 2009, 2010, 2011, 2012], limits=[2008, 2012])
    )
    return ppo

In [ ]:
wa_prepol_control = groupedby_df_washington[(groupedby_df_washington['Year']<2010)]
wa_postpol_control = groupedby_df_washington[(groupedby_df_washington['Year']>=2010)]

wa_prepol_treatment = groupedby_df_only_washington [(groupedby_df_only_washington['Year']<2010)]
wa_postpol_treatment = groupedby_df_only_washington[(groupedby_df_only_washington['Year']>=2010)]

In [ ]:
prepos = (
    pre_post(wa_prepol_treatment, wa_postpol_treatment, "Year", "MME", 2010)
    + labs(
        title="Opioid Shipments in Morphine Gram Equivalent rate for Washington Before and After 2010"
    )
    + ylab("Morphine Gram Equivalent rate")
)

print(prepos)

In [ ]:
diff = (
    diffIndiff(
        wa_prepol_control, wa_postpol_control, wa_prepol_treatment, wa_postpol_treatment, "Year", "MME", 2010
    )
    + labs(
        title="Opioid Shipments in Morphine Gram Equivalent rate for Washington vs. Control States",
        color="Policy_Change",
    )
    + ylab("Morphine Gram Equivalent rate")
)
print(diff)

In [ ]:
fl_prepol_control = groupedby_df_florida[(groupedby_df_florida['Year']<2010)]
fl_prepol_treatment = groupedby_df_only_florida [(groupedby_df_only_florida['Year']<2010)]
fl_postpol_control = groupedby_df_florida[(groupedby_df_florida['Year']>=2010)]
fl_postpol_treatment = groupedby_df_only_florida[(groupedby_df_only_florida['Year']>=2010)]

In [ ]:
prepos = (
    pre_post(fl_prepol_treatment, fl_postpol_treatment, "Year", "MME", 2010)
    + labs(
        title="Opioid Shipments in Morphine Gram Equivalent rate for Florida Before and After 2010"
    )
    + ylab("Morphine Gram Equivalent rate")
)

print(prepos)

In [ ]:
diff = (
    diffIndiff(
        fl_prepol_control, fl_postpol_control, fl_prepol_treatment, fl_postpol_treatment, "Year", "MME", 2010
    )
    + labs(
        title="Opioid Shipments in Morphine Gram Equivalent rate for Florida vs. Control States",
        color="Policy_Change",
    )
    + ylab("Morphine Gram Equivalent rate")
)
print(diff)

In [ ]:
tx_prepol_control = groupedby_df_texas[(groupedby_df_texas['Year']<2010)]
tx_prepol_treatment = groupedby_df_only_texas [(groupedby_df_only_texas['Year']<2010)]
tx_postpol_control = groupedby_df_texas[(groupedby_df_texas['Year']>=2010)]
tx_postpol_treatment = groupedby_df_only_texas[(groupedby_df_only_texas['Year']>=2010)]

In [ ]:
prepos = (
    pre_post(tx_prepol_treatment, tx_postpol_treatment, "Year", "MME", 2010)
    + labs(
        title="Opioid Shipments in Morphine Gram Equivalent rate for Washington Before and After 2010"
    )
    + ylab("Morphine Gram Equivalent rate")
)

print(prepos)

In [ ]:
diff = (
    diffIndiff(
        tx_prepol_control, tx_postpol_control, tx_prepol_treatment, tx_postpol_treatment, "Year", "MME", 2010
    )
    + labs(
        title="Opioid Shipments in Morphine Gram Equivalent rate for Texas vs. Control States",
        color="Policy_Change",
    )
    + ylab("Morphine Gram Equivalent rate")
)
print(diff)